In [ ]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()


In [1]:
# ==============================
# Imports and CUDA check
# ==============================
import torch
import pandas as pd
import random
import numpy as np
import ast
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from torch.utils.data import Sampler

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ==============================
# Seeds
# ==============================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# BalancedBatchSampler for QA (based on label)
# ==============================
# class BalancedBatchSampler(Sampler):
#     def __init__(self, dataset, batch_size):
#         self.dataset = dataset
#         self.labels = [ex.label for ex in dataset]
#         self.classes = list(sorted(set(self.labels)))
#         self.num_classes = len(self.classes)
#         assert batch_size % self.num_classes == 0, "Batch size must be divisible by number of classes"
#         self.batch_size_per_class = batch_size // self.num_classes

#     def __iter__(self):
#         class_indices = {c: np.where(np.array(self.labels) == c)[0].tolist() for c in self.classes}
#         for c in self.classes:
#             np.random.shuffle(class_indices[c])

#         num_batches = min(len(class_indices[c]) // self.batch_size_per_class for c in self.classes)

#         for i in range(num_batches):
#             batch = []
#             for c in self.classes:
#                 start = i * self.batch_size_per_class
#                 end = start + self.batch_size_per_class
#                 batch.extend(class_indices[c][start:end])
#             np.random.shuffle(batch)
#             yield batch

#     def __len__(self):
#         return min(len(np.where(np.array(self.labels) == c)[0]) // self.batch_size_per_class for c in self.classes)



# class QABalancedBatchSampler(Sampler):
#     def __init__(self, dataset, batch_size, options_per_question=4):
#         """
#         dataset: list of InputExample
#         batch_size: total examples per batch (must be multiple of options_per_question)
#         options_per_question: number of options per question (usually 4)
#         """
#         self.dataset = dataset
#         self.batch_size = batch_size
#         self.options_per_question = options_per_question
        
#         assert batch_size % options_per_question == 0, "Batch size must be divisible by options per question"
#         self.questions_per_batch = batch_size // options_per_question
        
#         # Group examples by question id
#         self.question_dict = {}
#         for ex in dataset:
#             if ex.qid not in self.question_dict:
#                 self.question_dict[ex.qid] = []
#             self.question_dict[ex.qid].append(ex)
        
#         self.qids = list(self.question_dict.keys())
        
#     def __iter__(self):
#         qids = self.qids.copy()
#         np.random.shuffle(qids)
        
#         # Split qids into batches
#         for i in range(0, len(qids), self.questions_per_batch):
#             batch_qids = qids[i:i+self.questions_per_batch]
#             batch = []
#             for qid in batch_qids:
#                 batch.extend(self.question_dict[qid])  # include all options
#             yield [self.dataset.index(ex) for ex in batch]  # yield indices
    
#     def __len__(self):
#         return len(self.qids) // self.questions_per_batch



class QABalancedBatchSampler(Sampler):
    def __init__(self, dataset, guid_to_qid, batch_size, options_per_question=4):
        self.dataset = dataset
        self.guid_to_qid = guid_to_qid
        self.batch_size = batch_size
        self.options_per_question = options_per_question
        assert batch_size % options_per_question == 0, "Batch size must be divisible by options per question"
        self.questions_per_batch = batch_size // options_per_question

        # Group examples by qid
        self.question_dict = {}
        for ex in dataset:
            qid = self.guid_to_qid[ex.guid]
            if qid not in self.question_dict:
                self.question_dict[qid] = []
            self.question_dict[qid].append(ex)

        self.qids = list(self.question_dict.keys())

    def __iter__(self):
        qids = self.qids.copy()
        np.random.shuffle(qids)
        for i in range(0, len(qids), self.questions_per_batch):
            batch_qids = qids[i:i+self.questions_per_batch]
            batch = []
            for qid in batch_qids:
                batch.extend(self.question_dict[qid])
            yield [self.dataset.index(ex) for ex in batch]

    def __len__(self):
        return len(self.qids) // self.questions_per_batch



# Raw data: first 16 questions
data_16 = [
    {
        "question": "ایک بیٹری کس چیز کو چلانے کے لیے کیمیائی توانائی کو برقی توانائی میں تبدیل کرتی ہے؟",
        "choices": np.array(["اسفنج", "پتھر", "اسٹیپلر", "کیلکولیٹر"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "کیا تمام حشرات کو مکمل بالغ ہونے سے پہلے تبدیلی کے ہر مرحلے سے گزرنا پڑتا ہے؟",
        "choices": np.array(["یہ تمام", "حشرات زندہ پیدا ہوتے ہیں",
                             "پیوپا کا مرحلہ کبھی کبھی چھوڑ دیا جاتا ہے",
                             "پیوپا ایک لازمی مرحلہ ہے"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "C"
    },
    {
        "question": "کون سی توانائی ماحول کے لیے سب سے زیادہ موزوں ہے؟",
        "choices": np.array(["کوئلہ", "پیٹرولیم", "قدرتی گیس", "سورج کی روشنی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "ایک مگرمچھ کیا کھائے گا؟",
        "choices": np.array(["مکڑی", "کیکڑا", "سکویڈ", "تلاپیا"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "حرارت کا ذریعہ ہو سکتا ہے",
        "choices": np.array(["ناک رگڑنا", "انٹارکٹیکا میں تیراکی کرنا", "برف کو چھونا",
                             "فریزر میں بیٹھنا"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "فنچوں کو پسند ہے",
        "choices": np.array(["لطیفے", "جادو", "ایوری ون لوز ریمنڈ", "میٹھے سلطان"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "روشنی کس چیز میں سے نہیں گزر سکتی؟",
        "choices": np.array(["لکڑی", "پانی", "ہوا", "شیشہ"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "ملچ کا ایک ڈھیر گل سڑ گیا ہے، اور جب اس کا سبب تلاش کیا جاتا ہے تو ایک باغبان کو نظر آتے ہیں",
        "choices": np.array(["بل کھاتے ہوئے جاندار", "لکھنے والے جاندار", "بڑے درخت", "جنگل کی آگ"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "چاند طلوع ہوتا ہے",
        "choices": np.array(["ایک دہائی میں ایک بار", "سال میں 4 بار", "ہفتے میں 7 بار", "مہینے میں 1 بار"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "C"
    },
    {
        "question": "پہاڑ کی ڈھلوان پر چٹانوں کے ٹوٹنے کی سب سے زیادہ ممکنہ وجہ کیا ہے؟",
        "choices": np.array(["صنوبر کے درخت کی جڑیں", "حقیقی عرفان", "سمندری خارپشت", "ہیوی میٹل موسیقی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "کچھ پرندے مقامات تلاش کرتے ہیں",
        "choices": np.array(["زمینی نشانیوں سے", "سڑک کے نشانات سے", "انڈوں سے", "مقناطیسی نمونوں سے"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "کس سے بیج پھیلنے کا امکان ہے؟",
        "choices": np.array(["ایک کار", "سورج کی ایک کرن", "ایک وہیل", "ایک ہمنگ برڈ"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "ایک کار جو ایک ایسے وسیلے پر چلتی ہے جو بالآخر ختم ہو جائے گا",
        "choices": np.array(["ڈائنوسار کی باقیات", "پانی کی طاقت", "شمسی توانائی", "بجلی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "بجلی کے کنڈکٹرز ان طریقوں سے کام کرتے ہیں جیسے",
        "choices": np.array(["مائیکروویو کو چلانے کے لیے ساکٹ میں پلگ لگانا",
                             "صبح کے وقت کافی پاٹ آن کرنا",
                             "استعمال کے بعد اس بات کو یقینی بنانا کہ ہیئر ڈرائر ان پلگ ہے",
                             "کمرے کی لائٹیں جلانا"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "اگر کسی پرندے کی چونچ اپنے ساتھیوں سے بڑی ہے، تو غالب امکان ہے کہ اس کے تھے",
        "choices": np.array(["بغیر چونچ والی ماں", "لمبی سونڈ والا باپ",
                             "چھوٹی چونچوں والے والدین", "ایسی ہی چونچوں والے آباؤ اجداد"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "اگر کوئی شخص روزانہ منکے بیچ رہا ہے اور پھر کوئی بھی مزید منکے نہیں خریدتا، تو بیچنے والے کا کیا ہوگا؟",
        "choices": np.array(["وہ زیادہ پیسے کمائے گا", "اس کی آمدنی رک جائے گی",
                             "اس کا منافع بڑھ جائے گا", "اس کی آمدنی زیادہ ہو جائے گی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "B"
    }
]

# Convert to OpenPrompt InputExample format
# train_dataset = []
# guid = 0

# for idx, item in enumerate(data_16):
#     question = item["question"]
#     labels = item["labels"]
#     choices = item["choices"]
#     correct_answer = item["answer"]

#     for label, choice in zip(labels, choices):
#         train_dataset.append(
#             InputExample(
#                 guid=f"q{idx}_{label}",
#                 text_a=question,
#                 text_b=choice,
#                 label=1 if label == correct_answer else 0,
#                 qid=f"q{idx}"   # <<< add this line
#             )
#         )
#         guid += 1


# Create InputExamples and qid mapping
train_dataset = []
guid = 0
guid_to_qid = {}  # map example guid to question id

for idx, item in enumerate(data_16):
    question = item["question"]
    labels = item["labels"]
    choices = item["choices"]
    correct_answer = item["answer"]
    qid = f"q{idx}"

    for label, choice in zip(labels, choices):
        ex = InputExample(
            guid=f"{qid}_{label}",
            text_a=question,
            text_b=choice,
            label=1 if label == correct_answer else 0
        )
        train_dataset.append(ex)
        guid_to_qid[ex.guid] = qid
        guid += 1




# # Quick stats
# print("Total examples:", len(train_dataset))
# print("Correct examples:", sum(ex.label==1 for ex in train_dataset))
# print("Incorrect examples:", sum(ex.label==0 for ex in train_dataset))

# train_dataset = []
# for idx, item in enumerate(data_16):
#     q = item["question"]
#     choices = item["choices"]
#     labels = item["labels"]
#     answer = item["answer"]
#     for l, c in zip(labels, choices):
#         train_dataset.append(InputExample(
#             guid=f"q{idx}_{l}",
#             text_a=q,
#             text_b=c,
#             label=1 if l == answer else 0
#         ))

# ==============================
# Classes
# ==============================
classes = ["incorrect", "correct"]
label_map = {"incorrect":0, "correct":1}

# ==============================
# Load PLM + Tokenizer
# ==============================
# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")



template = ManualTemplate(
        text='سوال: {"placeholder":"text_a"} جواب: {"placeholder":"text_b"} یہ جواب {"mask"} ہے۔',
        tokenizer=tokenizer,
)





templates = [
    # First 5: mask in the middle
    ("P1", ManualTemplate(
        text='سوال: {"placeholder":"text_a"} جواب: {"placeholder":"text_b"} یہ جواب {"mask"} ہے۔',
        tokenizer=tokenizer
    )),
    ("P2", ManualTemplate(
        text='{"placeholder":"text_a"} کے لیے جواب {"placeholder":"text_b"} صحیح طور پر {"mask"} ہے۔',
        tokenizer=tokenizer
    )),
    ("P3", ManualTemplate(
        text='مندرجہ ذیل سوال: {"placeholder":"text_a"} انتخاب: {"placeholder":"text_b"} نتیجہ {"mask"} ہونا چاہیے۔',
        tokenizer=tokenizer
    )),
    ("P4", ManualTemplate(
        text='سوال: {"placeholder":"text_a"} اور آپشن: {"placeholder":"text_b"} درست جواب {"mask"} ہے۔',
        tokenizer=tokenizer
    )),
    ("P5", ManualTemplate(
        text='{"placeholder":"text_a"} کے سوال کے لیے انتخاب {"placeholder":"text_b"} جواب {"mask"} ہے۔',
        tokenizer=tokenizer
    )),

    # Last 5: mask at the end
    ("P6", ManualTemplate(
        text='سوال: {"placeholder":"text_a"} جواب: {"placeholder":"text_b"} یہ جواب ہے {"mask"}',
        tokenizer=tokenizer
    )),
    ("P7", ManualTemplate(
        text='{"placeholder":"text_a"} کے لیے آپشن: {"placeholder":"text_b"} صحیح جواب {"mask"} ہے۔',
        tokenizer=tokenizer
    )),
    ("P8", ManualTemplate(
        text='سوال: {"placeholder":"text_a"} انتخاب: {"placeholder":"text_b"} نتیجہ {"mask"}',
        tokenizer=tokenizer
    )),
    ("P9", ManualTemplate(
        text='{"placeholder":"text_a"} کے سوال کا جواب {"placeholder":"text_b"} نتیجہ {"mask"} ہے۔',
        tokenizer=tokenizer
    )),
    ("P10", ManualTemplate(
        text='{"placeholder":"text_a"} اور انتخاب: {"placeholder":"text_b"} صحیح جواب {"mask"}',
        tokenizer=tokenizer
    )),
]


# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
    "correct": ["درست", "صحیح"], 
    "incorrect": ["غلط", "نا درست"]
    },
    tokenizer=tokenizer,
)


# ==============================
# Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,  # initial template
    plm=plm,
    verbalizer=verbalizer
)
prompt_model.to(device)




C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized fr

PromptForClassification(
  (prompt_model): PromptModel(
    (plm): BertForMaskedLM(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
 

In [2]:
# ==============================
# Training loop with multi-template switching + QABalancedBatchSampler
# ==============================
T = 20       # epochs
K = 1        # steps per template
batch_size = 16  # total examples per batch (must be multiple of 4)
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

prompt_model.train()
all_epoch_patterns = {}

for epoch in range(T):
    print(f"\n🟦 Epoch {epoch+1}/{T}")

    torch.cuda.empty_cache()

    # Random initial template
    prompt_name, current_template = random.choice(templates)
    epoch_pattern = []

    # QABalancedBatchSampler ensures full questions per batch
    # sampler = QABalancedBatchSampler(train_dataset, batch_size=batch_size, options_per_question=4)
    sampler = QABalancedBatchSampler(train_dataset, guid_to_qid=guid_to_qid, batch_size=batch_size, options_per_question=4)
    train_loader = PromptDataLoader(
        dataset=train_dataset,
        tokenizer=tokenizer,
        template=current_template,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=128,
        batch_size=batch_size,
        batch_sampler=sampler,
        shuffle=False
    )

    # # 🔹 Debug: check what the loader yields
    # for batch_idx, batch in enumerate(train_loader):
    #     print(batch_idx, batch.keys(), len(batch['input_ids']))
    #     break  # just check the first batch

    
    step_counter = 0
    epoch_loss = 0.0

    for batch in train_loader:
        # Only move tensor entries to device
        batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}
        
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
    
        epoch_loss += loss.item()
        epoch_pattern.append(prompt_name)
    
        step_counter += 1
    
        # Switch template every K steps
        if step_counter % K == 0:
            prompt_name, current_template = random.choice(templates)
    
            train_loader = PromptDataLoader(
                dataset=train_dataset,
                tokenizer=tokenizer,
                template=current_template,
                tokenizer_wrapper_class=WrapperClass,
                max_seq_length=128,
                batch_size=batch_size,
                batch_sampler=sampler,
                shuffle=False
            )


    all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
    print(f"Prompt pattern: {epoch_pattern}")



🟦 Epoch 1/20


tokenizing: 64it [00:00, 2233.35it/s]
tokenizing: 64it [00:00, 1783.00it/s]
tokenizing: 64it [00:00, 834.66it/s]
tokenizing: 64it [00:00, 2979.01it/s]
tokenizing: 64it [00:00, 1930.37it/s]


Epoch 1 Loss: nan
Prompt pattern: ['P2', 'P1', 'P5', 'P4']

🟦 Epoch 2/20


tokenizing: 64it [00:00, 1763.68it/s]
tokenizing: 64it [00:00, 2622.29it/s]
tokenizing: 64it [00:00, 2134.40it/s]
tokenizing: 64it [00:00, 1913.61it/s]
tokenizing: 64it [00:00, 2481.06it/s]


Epoch 2 Loss: nan
Prompt pattern: ['P3', 'P2', 'P9', 'P2']

🟦 Epoch 3/20


...

Epoch 18 Loss: nan
Prompt pattern: ['P1', 'P6', 'P7', 'P5']

🟦 Epoch 19/20


tokenizing: 64it [00:00, 653.67it/s]
tokenizing: 64it [00:00, 2462.58it/s]
tokenizing: 64it [00:00, 882.36it/s]
tokenizing: 64it [00:00, 2322.31it/s]
tokenizing: 64it [00:00, 2376.44it/s]


Epoch 19 Loss: nan
Prompt pattern: ['P4', 'P10', 'P6', 'P4']

🟦 Epoch 20/20


tokenizing: 64it [00:00, 1598.35it/s]
tokenizing: 64it [00:00, 2905.71it/s]
tokenizing: 64it [00:00, 1757.13it/s]
tokenizing: 64it [00:00, 1521.22it/s]
tokenizing: 64it [00:00, 1357.69it/s]

Epoch 20 Loss: nan
Prompt pattern: ['P7', 'P8', 'P3', 'P5']


In [3]:
# ==============================
# Load evaluation dataset
# ==============================
csv_path = r"C:\Users\stdFurqan\Desktop\paft\QA\openbookqa_urdu_test_cleaned.csv"
df = pd.read_csv(csv_path)

eval_dataset = []
eval_qids = []   # keeps question ids aligned with examples

for i, row in df.iterrows():
    q = row['urdu_question_stem']
    choices_dict = eval(row['clean_choices'])
    labels = choices_dict['label']
    texts = choices_dict['text']
    answer = row['answerKey']

    for l, t in zip(labels, texts):
        eval_dataset.append(
            InputExample(
                guid=f"eval_{i}_{l}",
                text_a=q,
                text_b=t,
                label=1 if l == answer else 0
            )
        )
        eval_qids.append(i)   # question id


eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,   # single template OR loop over templates
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

prompt_model.eval()

question_scores = {}   # qid -> list of (score, label)

with torch.no_grad():
    example_idx = 0

    for batch in eval_loader:
        batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}

        logits = prompt_model(batch)          # [B, 2]
        pos_scores = logits[:, 1]              # score for "correct" class

        for score, label in zip(pos_scores.cpu(), batch['label'].cpu()):
            qid = eval_qids[example_idx]

            if qid not in question_scores:
                question_scores[qid] = []

            question_scores[qid].append((score.item(), label.item()))
            example_idx += 1


correct = 0
total = 0

for qid, options in question_scores.items():
    # pick option with highest score
    pred_score, pred_label = max(options, key=lambda x: x[0])

    if pred_label == 1:
        correct += 1
    total += 1

qa_accuracy = correct / total
print(f"\n📊 QA Accuracy: {qa_accuracy:.4f}")


tokenizing: 3968it [00:02, 1565.93it/s]



📊 QA Accuracy: 0.2692
